# Семинар 2. Текстовая регрессия

В этом семинаре нам предстоит решить задачу предсказания зарплаты по описанию вакансии.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Для начала скачиваем данные.

In [ ]:
!wget https://ysda-seminars.s3.eu-central-1.amazonaws.com/Train_rev1.zip
data = pd.read_csv("./Train_rev1.zip", compression='zip', index_col=None)
data.shape

In [ ]:
data.head()

Первая проблема, с которой мы столкнемся - это распределение зарплат, близкое к экспоненциальному. Многие получают среднюю зарплату, но есть небольшая кучка людей с аномально высокой зарплатой. Как вы знаете из курса по машинному обучению, обучение модели на минимизацию MSE имеет смысл только тогда, когда таргеты распределены нормально. В противном случае стоит либо использовать другую функцию ошибки, либо как-то превести данные к нормальному распределению. Мы пойдем по второму пути и накинем логарифм на значения зарплат.

_Больше информации по этому вопросу в [описании соревнования](https://www.kaggle.com/c/job-salary-prediction#description)._

In [ ]:
data['Log1pSalary'] = np.log1p(data['SalaryNormalized']).astype('float32')

plt.figure(figsize=[14, 4])
plt.subplot(1, 2, 1)
plt.hist(data["SalaryNormalized"], bins=50);

plt.subplot(1, 2, 2)
plt.hist(data['Log1pSalary'], bins=50);

Итак, наша задача - предсказать значение __Log1pSalary__.

Для этого мы можем извлечь признаки из
* текстовых данных: __`Title`__,  __`FullDescription`__
* категориальных данных: __`Category`__, __`Company`__, __`LocationNormalized`__, __`ContractType`__, __`ContractTime`__

In [ ]:
text_columns = ["Title", "FullDescription"]
categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]
TARGET_COLUMN = "Log1pSalary"

data[categorical_columns] = data[categorical_columns].fillna('NaN') # cast missing values to string "NaN"

data.sample(3)

### Начнем с текстовых данных

Как и в любой NLP задаче, сперва текст надо токенизировать. Сделаем это так же, как и на прошлом семинаре. Приведем текст к нижнему регистру, удалим всю пунктуацию и поделим на слова.

Для дальнейшего удобства лучше хранить каждый токенизированных текст в виде __одной строки со словами, разделенными пробелами__, а не списка, как было раньше.

In [ ]:
print("Raw text:")
print(data["FullDescription"][2::100000])

In [ ]:
import re

def tokenize(text):
    reg = re.compile(r'\w+')
    return reg.findall(text.lower())

In [ ]:
data["Title"] = # your code here
data["FullDescription"] = # your code here

In [ ]:
print("Tokenized:")
print(data["FullDescription"][2::100000])
assert data["FullDescription"][2][:50] == 'mathematical modeller simulation analyst operation'
assert data["Title"][54321] == 'international digital account manager german'

Не все слова одинаково полезны. Некоторые из них встречаются очень редко, и поэтому модель не сможет выучить их значение. При этом эмбеддинги для таких слов к тому же будут занимать место в памяти, что плохо.

Давайте посчитаем, сколько раз встречалось каждое слово, и выкинем редкие.

In [ ]:
from collections import Counter
token_counts = Counter()

# Count how many times does each token occur in both "Title" and "FullDescription" in total
# your code here

In [ ]:
print("Total unique tokens :", len(token_counts))
print('\n'.join(map(str, token_counts.most_common(n=5))))
print('...')
print('\n'.join(map(str, token_counts.most_common()[-3:])))

assert token_counts.most_common(1)[0][1] in  range(2600000, 2700000)
assert len(token_counts) in range(200000, 210000)
print('Correct!')

In [ ]:
# Let's see how many words are there for each count
plt.hist(list(token_counts.values()), range=[0, 10**4], bins=50, log=True)
plt.xlabel("Word counts")
plt.show()

Создайте список уникальных токенов, которые встречались не меньше 10 раз, и отсортируйте их по частоте встречания.

In [ ]:
min_count = 10

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = # your code here

# Add a special tokens for unknown and empty words
UNK, PAD = "UNK", "PAD"
tokens = [UNK, PAD] + tokens

In [ ]:
print("Vocabulary size:", len(tokens))
assert type(tokens) == list
assert len(tokens) in range(32000, 35000)
assert 'me' in tokens
assert UNK in tokens
print("Correct!")

Создайте словарь _\{токен: индекс\}_ для всех токенов из списка.

In [ ]:
token_to_id = # your code here

In [ ]:
assert isinstance(token_to_id, dict)
assert len(token_to_id) == len(tokens)
for tok in tokens:
    assert tokens[token_to_id[tok]] == tok

print("Correct!")

Эта функция кодирует набор текстов в виде матрицы из индексов токенов, добавляя паддинги к коротким текстам.

In [ ]:
UNK_IDX, PAD_IDX = token_to_id[UNK], token_to_id[PAD]


def as_matrix(sequences, max_len=None):
    """ Convert a list of tokens into a matrix with padding """
    if isinstance(sequences[0], str):
        sequences = [x.split() for x in sequences]

    max_sequence_len = max([len(x) for x in sequences])
    if max_len is not None and max_len < max_sequence_len:
        max_sequence_len = max_len

    matrix = np.full((len(sequences), max_sequence_len), np.int32(PAD_IDX))
    for i, seq in enumerate(sequences):
        row_ix = [token_to_id.get(word, UNK_IDX) for word in seq[:max_sequence_len]]
        matrix[i, :len(row_ix)] = row_ix

    return matrix

In [ ]:
print("Lines:")
print('\n'.join(data["Title"][::100000].values), end='\n\n')
print("Matrix:")
print(as_matrix(data["Title"][::100000]))

### Категориальные данные

Для кодирования категориальных признаков мы будем использовать one-hot-encoding. Это не лучшая идея в данном случае, так как различных значений у признаков много. Однако, реализацию более сложных методов мы оставим для вас.


In [ ]:
from sklearn.feature_extraction import DictVectorizer

# we only consider top-1k most frequent companies to minimize memory usage
top_companies, top_counts = zip(*Counter(data['Company']).most_common(1000))
recognized_companies = set(top_companies)
data["Company"] = data["Company"].apply(lambda comp: comp if comp in recognized_companies else "Other")

categorical_vectorizer = DictVectorizer(dtype=np.float32, sparse=False)
categorical_vectorizer.fit(data[categorical_columns].apply(dict, axis=1))

In [ ]:
data[categorical_columns].apply(dict, axis=1)[0]

### Обучение модели

Мы проделали все подготовительные шаги, а значит, самое время обучить нейронную сеть.

Стоит отметить, что мы немного схалтурили. Во-первых, мы не будем выделять валидационную выборку, так как не будем особо стараться улучшать модель. Во-вторых, для токенизации и сбора словаря мы использовали все данные, а не только тренировочные, как это надо делать. При желании, вы можете поправить эти неточности и посмотреть, как будет меняться результат.

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)
data_train.index = range(len(data_train))
data_val.index = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

In [ ]:
import torch
import torch.nn as nn
import torch.functional as F


device = 'cuda' if torch.cuda.is_available() else 'cpu'


def to_tensors(batch, device):
    batch_tensors = dict()
    for key, arr in batch.items():
        if key in ["FullDescription", "Title"]:
            batch_tensors[key] = torch.tensor(arr, device=device, dtype=torch.int64)
        else:
            batch_tensors[key] = torch.tensor(arr, device=device)
    return batch_tensors


def apply_word_dropout(matrix, keep_prop, replace_with=UNK_IDX, pad_ix=PAD_IDX,):
    dropout_mask = np.random.choice(2, np.shape(matrix), p=[keep_prop, 1 - keep_prop])
    dropout_mask &= matrix != pad_ix
    return np.choose(dropout_mask, [matrix, np.full_like(matrix, replace_with)])


def make_batch(data, max_len=None, word_dropout=0, device=device):
    """
    Creates a keras-friendly dict from the batch data.
    :param word_dropout: replaces token index with UNK_IDX with this probability
    :returns: a dict with {
        'Title' : int64[batch, title_max_len],
        'FullDescription' : int64[batch, descr_max_len],
        'Categorical' : float32[batch, ohe_len],
        'Log1pSalary' : float32[batch]
    }
    """
    batch = {}
    batch["Title"] = as_matrix(data["Title"].values, max_len)
    batch["FullDescription"] = as_matrix(data["FullDescription"].values, max_len)
    batch['Categorical'] = categorical_vectorizer.transform(data[categorical_columns].apply(dict, axis=1))

    if word_dropout > 0:
        batch["FullDescription"] = apply_word_dropout(batch["FullDescription"], 1. - word_dropout)

    if TARGET_COLUMN in data.columns:
        batch[TARGET_COLUMN] = data[TARGET_COLUMN].values

    return to_tensors(batch, device)

In [ ]:
make_batch(data_train[:3], max_len=10)

#### Архитектура

Наша модель будет состоять из трех веток:
* Кодировщик заголовка
* Кодировщик описания
* Кодировщик категориальных признаков

Выходы всех трех веток будут конкатенироваться и затем преобразовываться в скаляр с помощью полносвязного слоя.

![scheme](https://github.com/yandexdataschool/nlp_course/raw/master/resources/w2_conv_arch.png)

In [ ]:
class SalaryPredictor(nn.Module):
    def __init__(self, n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_), hid_size=64):
        super().__init__()
        # your code here

    def forward(self, batch):
        pass
        # your code here

In [ ]:
model = SalaryPredictor().to(device)
batch = make_batch(data_train[:100])
criterion = nn.MSELoss()

dummy_pred = model(batch)
dummy_loss = criterion(dummy_pred, batch[TARGET_COLUMN])
assert dummy_pred.shape == torch.Size([100])
assert len(torch.unique(dummy_pred)) > 20, "model returns suspiciously few unique outputs. Check your initialization"
assert dummy_loss.ndim == 0 and 0. <= dummy_loss <= 250., "make sure you minimize MSE"

#### Обучение и валидация

Создадим функцию для бесконечной подачи батчей данных.

In [ ]:
def iterate_minibatches(data, batch_size=256, shuffle=True, cycle=False, device=device):
    """ iterates minibatches of data in random order """
    while True:
        indices = np.arange(len(data))
        if shuffle:
            indices = np.random.permutation(indices)

        for start in range(0, len(indices), batch_size):
            batch = make_batch(data.iloc[indices[start : start + batch_size]], device=device)
            yield batch

        if not cycle:
            break

Во время обучения мы будем логировать не только значение ошибки на валидации (зарплата у нас в логарифмической шкале, поэтому такую ошибку сложно интерпретировать), но и стандартное отклонение ошибки предсказания зарплаты в долларах.

In [ ]:
from tqdm.auto import tqdm

BATCH_SIZE = 16
EPOCHS = 5

In [ ]:
@torch.no_grad()
def print_metrics(model, data, batch_size=BATCH_SIZE, name='', device=torch.device('cpu')):
    squared_error = mse_exp_loss = num_samples = 0.0
    model.eval()

    mse_loss = nn.MSELoss(reduction='sum')

    for batch in iterate_minibatches(data, batch_size=batch_size, shuffle=False, device=device):
        batch_pred = model(batch)
        squared_error += mse_loss(batch_pred, batch[TARGET_COLUMN]).item()
        mse_exp_loss += mse_loss(torch.expm1(batch_pred), torch.expm1(batch[TARGET_COLUMN])).item()
        num_samples += len(batch_pred)

    mse = squared_error / num_samples
    salary_pred_std = mse_exp_loss**0.5 / num_samples

    print(f'Mean square error: {round(mse, 4)}')
    print(f'Std of salary prediction error: {round(salary_pred_std, 4)}')

    return mse, salary_pred_std

### Обучение модели

Тут мы снова не будем слишком избирательны при выборе оптимизатора, расписания скорости обучения и гиперпараметров. Возьмем все самое простое, а при желании вы можете попробовать улучшить наш результат.

In [ ]:
model = SalaryPredictor().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for epoch in range(EPOCHS):
    print(f'epoch: {epoch}')
    model.train()
    for i, batch in tqdm(enumerate(
            iterate_minibatches(data_train, batch_size=BATCH_SIZE, device=device)),
            total=len(data_train) // BATCH_SIZE
        ):
        pred = model(batch)
        loss = criterion(pred, batch[TARGET_COLUMN])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print_metrics(model, data_val, device=device)

### Объяснение предсказаний

На практике очень полезно бывает уметь объяснять, почему модель выдала то или иное предсказание. Для простейших моделей (линейныя регрессия, дерево решений) возможность интерпретации встроена по умолчанию. Однако в случае нейронных сетей все не так просто, потому что зависимость предсказаний от признаков имеет очень сложную структуру, поэтому приходится придумывать всякие трюки.

Вот некоторые из способов посмотреть внутрь модели:
* Изменить вход и посмотреть, как меняется предсказание
* Поискать примеры, которые максимизируют или минимизируют активации конкретных нейронов (_больше об этом на [distill.pub](https://distill.pub/2018/building-blocks/)_)
* Построить локальную _линейую_ аппроксимацию модели: [LIME article](https://arxiv.org/abs/1602.04938), [eli5 library](https://github.com/TeamHG-Memex/eli5/tree/master/eli5/formatters)
* Обучить нейронную сеть, сопоставляющую каждому слову его важность с точки зрения модели [L2X article](https://arxiv.org/abs/1802.07814)

В этом семинаре мы остановимся на первом способе, как на самом простом.

In [ ]:
@torch.no_grad()
def explain(model, sample, col_name='Title'):
    """ Computes the effect each word had on model predictions """

    baseline_pred = model(make_batch(sample, device=device)).cpu()
    sample = dict(sample.iloc[0])

    sample_col_tokens = [tokens[token_to_id.get(tok, 0)] for tok in sample[col_name].split()]
    data_drop_one_token = pd.DataFrame([sample] * (len(sample_col_tokens) + 1))

    for drop_i in range(len(sample_col_tokens)):
        data_drop_one_token.loc[drop_i, col_name] = ' '.join(UNK if i == drop_i else tok
                                                   for i, tok in enumerate(sample_col_tokens))

    predictions_drop_one_token = model(make_batch(data_drop_one_token, device=device)).cpu()
    diffs = baseline_pred - predictions_drop_one_token
    return list(zip(sample_col_tokens, diffs))

In [ ]:
from IPython.display import HTML, display_html


def draw_html(tokens_and_weights, cmap=plt.get_cmap("bwr"), display=True,
              token_template="""<span style="background-color: {color_hex}">{token}</span>""",
              font_style="font-size:14px;"
             ):

    def get_color_hex(weight):
        rgba = cmap(1. / (1 + np.exp(float(weight))), bytes=True)
        return '#%02X%02X%02X' % rgba[:3]

    tokens_html = [
        token_template.format(token=token, color_hex=get_color_hex(weight))
        for token, weight in tokens_and_weights
    ]


    raw_html = """<p style="{}">{}</p>""".format(font_style, ' '.join(tokens_html))
    if display:
        display_html(HTML(raw_html))

    return raw_html


In [ ]:
i = 36605
tokens_and_weights = explain(model, data.loc[i:i], "Title")
draw_html([(tok, weight * 5) for tok, weight in tokens_and_weights], font_style='font-size:20px;');

tokens_and_weights = explain(model, data.loc[i:i], "FullDescription")
draw_html([(tok, weight * 10) for tok, weight in tokens_and_weights]);

In [ ]:
i = 12010
tokens_and_weights = explain(model, data.loc[i:i], "Title")
draw_html([(tok, weight * 5) for tok, weight in tokens_and_weights], font_style='font-size:20px;');

tokens_and_weights = explain(model, data.loc[i:i], "FullDescription")
draw_html([(tok, weight * 10) for tok, weight in tokens_and_weights]);

In [ ]:
i = np.random.randint(len(data))
print("Index:", i)
with torch.no_grad():
    print("Salary (gbp):", np.expm1(model(make_batch(data.iloc[i: i+1], device=device)).cpu()))

tokens_and_weights = explain(model, data.loc[i:i], "Title")
draw_html([(tok, weight * 5) for tok, weight in tokens_and_weights], font_style='font-size:20px;');

tokens_and_weights = explain(model, data.loc[i:i], "FullDescription")
draw_html([(tok, weight * 10) for tok, weight in tokens_and_weights]);